In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='/media/bmn/NEMO/DATA/AI/numer.ai/'

In [10]:
train_df = pd.read_csv(f'{PATH}numerai_training_data.csv')
test_df = pd.read_csv(f'{PATH}numerai_tournament_data.csv')
sample_predictions_df = pd.read_csv(f'{PATH}example_predictions.csv')

In [4]:
sample_predictions_df.head()

,id,probability
0,n3e8c8e5628154b3,0.51029
1,nad9c8e8a783b485,0.48683
2,n9e1851afe65a4d8,0.50192
3,n6bd653c2e3d0464,0.51429
4,n660f94da36764f6,0.49451


In [17]:
from IPython.display import HTML

In [18]:
for t in [train_df, test_df]:
    display(t.head())

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
id,,,,,,,,,,,,,,,,,,,,,
n2b2e3dd163cb422,0.43487,0.44645,0.25802,0.37149,0.62235,0.67451,0.68103,0.45166,0.82443,0.38744,...,0.52962,0.42439,0.51680,0.46297,0.57426,0.57946,0.49646,0.48968,0.54194,1
n177021a571c94c8,0.50038,0.39216,0.38394,0.51213,0.36660,0.46911,0.68204,0.61320,0.39951,0.39705,...,0.51669,0.48445,0.57587,0.59860,0.67558,0.45577,0.80908,0.50287,0.61629,0
n7830fa4c0cd8466,0.47416,0.34143,0.39528,0.46337,0.72953,0.45962,0.47869,0.68118,0.57239,0.40318,...,0.41458,0.34804,0.29058,0.51382,0.36389,0.80602,0.39253,0.41821,0.58679,0
nc594a184cee941b,0.48759,0.55903,0.43987,0.38834,0.44650,0.46389,0.70749,0.63182,0.36395,0.45083,...,0.28776,0.42881,0.55402,0.53695,0.48793,0.62432,0.52898,0.49009,0.49557,0
n85e624e4714a4ca,0.62443,0.72606,0.74083,0.33016,0.48215,0.43777,0.63622,0.61748,0.40747,0.20916,...,0.40700,0.53875,0.45191,0.51101,0.55764,0.48148,0.77119,0.39730,0.51359,0


,id,era,data_type,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
0,n3e8c8e5628154b3,era121,validation,0.42057,0.69099,0.58832,0.36269,0.54931,0.74314,0.55197,...,0.56421,0.51444,0.56193,0.40427,0.67298,0.48918,0.49836,0.46838,0.47957,1.0
1,nad9c8e8a783b485,era121,validation,0.22094,0.38614,0.63439,0.66683,0.43546,0.45494,0.38885,...,0.56674,0.22710,0.37974,0.57428,0.38184,0.48088,0.46274,0.44765,0.70332,0.0
2,n9e1851afe65a4d8,era121,validation,0.45368,0.42176,0.48263,0.38216,0.53054,0.52949,0.48844,...,0.56735,0.37320,0.42111,0.52061,0.42437,0.50824,0.41084,0.42347,0.43478,0.0
3,n6bd653c2e3d0464,era121,validation,0.43173,0.44388,0.48123,0.44916,0.47387,0.54805,0.59780,...,0.65622,0.40034,0.45703,0.60313,0.65507,0.49361,0.49393,0.61086,0.44030,0.0
4,n660f94da36764f6,era121,validation,0.39561,0.33568,0.37945,0.47066,0.72195,0.50626,0.49405,...,0.37352,0.42203,0.37988,0.52681,0.34987,0.56615,0.39452,0.51381,0.48003,0.0


In [19]:
nt = len(test_df)
n = len(train_df)

In [20]:
contin_vars = []
for i in range(1, 51):
    contin_vars.append(f'feature{i}')
categorical_vars = ['era']

In [21]:
len(contin_vars), len(categorical_vars)

(50, 1)

In [22]:
dep = 'target'
train_df = train_df[categorical_vars+contin_vars+[dep, 'id']].copy()

KeyError: "['era' 'id'] not in index"

In [11]:
test_df[dep] = 0
test_df = test_df[categorical_vars+contin_vars+[dep, 'id']].copy()

In [12]:
for v in categorical_vars: train_df[v] = train_df[v].astype('category').cat.as_ordered()

In [13]:
apply_cats(test_df, train_df)

In [14]:
train_df.head()

,era,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target,id
0,era1,0.43487,0.44645,0.25802,0.37149,0.62235,0.67451,0.68103,0.45166,0.82443,...,0.42439,0.51680,0.46297,0.57426,0.57946,0.49646,0.48968,0.54194,1,n2b2e3dd163cb422
1,era1,0.50038,0.39216,0.38394,0.51213,0.36660,0.46911,0.68204,0.61320,0.39951,...,0.48445,0.57587,0.59860,0.67558,0.45577,0.80908,0.50287,0.61629,0,n177021a571c94c8
2,era1,0.47416,0.34143,0.39528,0.46337,0.72953,0.45962,0.47869,0.68118,0.57239,...,0.34804,0.29058,0.51382,0.36389,0.80602,0.39253,0.41821,0.58679,0,n7830fa4c0cd8466
3,era1,0.48759,0.55903,0.43987,0.38834,0.44650,0.46389,0.70749,0.63182,0.36395,...,0.42881,0.55402,0.53695,0.48793,0.62432,0.52898,0.49009,0.49557,0,nc594a184cee941b
4,era1,0.62443,0.72606,0.74083,0.33016,0.48215,0.43777,0.63622,0.61748,0.40747,...,0.53875,0.45191,0.51101,0.55764,0.48148,0.77119,0.39730,0.51359,0,n85e624e4714a4ca


In [15]:
for t in [train_df, test_df]: display(DataFrameSummary(t).summary())

,era,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target,id
count,NaN,393613,393613,393613,393613,393613,393613,393613,393613,393613,...,393613,393613,393613,393613,393613,393613,393613,393613,393613,NaN
mean,NaN,0.467463,0.457734,0.481866,0.444961,0.509518,0.484868,0.524472,0.558369,0.5175,...,0.483345,0.488708,0.532559,0.53034,0.519575,0.520685,0.494798,0.521452,0.499923,NaN
std,NaN,0.132239,0.0992284,0.116557,0.108713,0.113137,0.115371,0.123702,0.11497,0.101921,...,0.12068,0.135982,0.115896,0.107398,0.112527,0.116171,0.109163,0.107282,0.500001,NaN
min,NaN,0,0,0,0.01488,0,0.01171,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
25%,NaN,0.37472,0.39142,0.40229,0.37228,0.43495,0.40467,0.44128,0.48223,0.45102,...,0.39796,0.39524,0.45517,0.45631,0.44361,0.4411,0.42123,0.44985,0,NaN
50%,NaN,0.46674,0.45725,0.47961,0.44459,0.51256,0.48437,0.52745,0.56214,0.51993,...,0.47874,0.48575,0.53637,0.5285,0.51944,0.52026,0.49545,0.52216,0,NaN
75%,NaN,0.55886,0.52344,0.55935,0.51676,0.58712,0.56348,0.61054,0.63851,0.58638,...,0.56401,0.57976,0.61315,0.60285,0.59522,0.59969,0.56921,0.59398,1,NaN
max,NaN,1,0.93888,1,0.91131,1,1,0.9928,0.98644,0.96931,...,0.99827,0.99806,0.99359,1,0.98335,1,0.96184,1,1,NaN
counts,393613,393613,393613,393613,393613,393613,393613,393613,393613,393613,...,393613,393613,393613,393613,393613,393613,393613,393613,393613,393613
uniques,120,62899,50677,57469,54809,56186,56506,60106,56601,51924,...,58536,65248,56986,53471,55852,57059,54433,53917,2,393613


,era,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target,id
count,NaN,243232,243232,243232,243232,243232,243232,243232,243232,243232,...,243232,243232,243232,243232,243232,243232,243232,243232,243232,NaN
mean,NaN,0.468232,0.457441,0.481581,0.445015,0.510535,0.486294,0.524888,0.558997,0.517202,...,0.484655,0.489688,0.531092,0.531164,0.519129,0.519977,0.495059,0.522234,0,NaN
std,NaN,0.133113,0.0956802,0.11439,0.108526,0.113127,0.116107,0.121072,0.112877,0.10159,...,0.121532,0.133621,0.114716,0.105976,0.112735,0.115119,0.109512,0.104984,0,NaN
min,NaN,0.00039,0.03471,0.01711,0,0.01191,0,0.04828,0.05351,0.03589,...,0.04817,0.02911,0.01355,0.11461,0.01544,0.03223,0.05201,0.05444,0,NaN
25%,NaN,0.37501,0.393018,0.40289,0.372067,0.43549,0.40586,0.44349,0.48539,0.44981,...,0.39926,0.397667,0.453928,0.45745,0.44307,0.44084,0.42066,0.45194,0,NaN
50%,NaN,0.46666,0.45692,0.47921,0.44428,0.51434,0.48547,0.52847,0.564215,0.5183,...,0.47998,0.48644,0.53382,0.52811,0.51946,0.51877,0.4962,0.5237,0,NaN
75%,NaN,0.559972,0.52142,0.55801,0.516733,0.58855,0.56535,0.60936,0.63792,0.58513,...,0.565403,0.57823,0.61075,0.6028,0.59553,0.59799,0.570592,0.59382,0,NaN
max,NaN,0.97416,1,0.99326,1,0.98296,0.97156,1,1,1,...,1,1,1,0.99978,1,0.98554,1,0.99891,0,NaN
counts,0,243232,243232,243232,243232,243232,243232,243232,243232,243232,...,243232,243232,243232,243232,243232,243232,243232,243232,243232,243232
uniques,0,58022,45068,51814,49920,51441,52191,54069,51043,47422,...,54051,58523,51759,48543,51254,51875,49912,48426,1,243232


In [ ]:
for v in contin_vars:
    train_df[v] = train_df[v].astype('float32')
    test_df[v] = test_df[v].astype('float32')

In [ ]:
# sample for testing
idxs = get_cv_idxs(n, val_pct=150000/n)
train_samp = train_df.iloc[idxs].set_index("id")
samp_size = len(train_samp); samp_size

In [ ]:
# full_dataset
samp_size = n
train_samp = train_df.set_index("id")

In [ ]:
train_samp.tail(2)

In [ ]:
df, y, nas, mapper = proc_df(train_samp, 'target', do_scale=True)
yl = np.log(y)

In [ ]:
test_df = test_df.set_index("id")

In [ ]:
df_test, _, nas, mapper = proc_df(test_df, 'target', do_scale=True,
                                  mapper=mapper, na_dict=nas)

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [ ]:
y_range = (0, 1)

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=categorical_vars, bs=128,
                                       test_df=df_test)

In [ ]:
cat_sz = [(c, len(train_df[c].cat.categories)+1) for c in categorical_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(categorical_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot()

In [ ]:
m.fit(lr, 3, metrics=[accuracy])

In [ ]:
m.fit(lr, 6, cycle_len=2, cycle_mult=2, cycle_save_name=f'solenoid_{lr}', metrics=[accuracy])

In [ ]:
m.predict()

In [ ]:
test_df.headad()

# Random Forest Regressor

In [1]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [13]:
rf_train_df = train_df.drop('data_type', axis=1)
rf_train_df = train_df.drop('era', axis=1)

In [15]:
rf_train_df = train_df.set_index('id')

In [16]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(rf_train_df.drop('target', axis=1), rf_train_df.target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
m.score(df, dep)